In [1]:
import pandas as pd
import numpy as np
import os
import glob
from pprint import pprint
from itertools import islice
import random

# Set the max_columns option to None to display all columns
pd.set_option('display.max_columns', None)

In [2]:
# List of state abbreviations
state_abbreviations = ['AZ', 'CA', 'CO', 'ID', 'MT', 'NM', 'NV', 'OR', 'UT', 'WA', 'WY']

bea_df = pd.read_csv('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/BEA_economics_CAEMP25N_clean/BEA_combined_2001_2021.csv')
census_df = pd.read_csv('/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/census_clean/census_2001_2021_clean.csv')

print(len(bea_df))
print(len(census_df))

414
414


In [3]:
merge_df = pd.concat([census_df, bea_df], ignore_index=True)

In [4]:
len(merge_df)*2

1656

### Calculate the normalized jobs_per_100k people values per year

In [5]:
jobsper100k_new_rows = []  # Step 1: Create an empty list for new rows

# Set the maximum number of iterations you want
num_iterations = len(merge_df['geofips'].unique())  # Adjust this to your desired number of iterations

# Create an iterator over the unique 'geofips'
geofips_iterator = iter(merge_df['geofips'].unique())

# Step 2: Iterate through unique 'geofips'
for geofips in islice(geofips_iterator, num_iterations):
    # Extract data for the current 'geofips'
    subset = merge_df[merge_df['geofips'] == geofips]
#     print(f"Temporary df for geofips {geofips}:")
#     print("***///***")

    # Create a new row with the calculated values
    new_row = subset.iloc[0].copy() # copies one of the existing rows for this geofips

    new_row['description'] = 'jobs_per_100k' # updates the description column
    new_row['linecode'] = 30.0  # updates the linecode    

    # Calculate 'jobs_per_100k' for each year
    for year in range(2001, 2022):
        column = f'yr_{year}'
        linecode_jobs = 10.0
        linecode_population = 20.0


        # Calculate 'jobs_per_100k' for this year
        jobs_data = subset[subset['linecode'] == linecode_jobs][column].values[0]
        population_data = subset[subset['linecode'] == linecode_population][column].values[0]

        jobs_per_100k = round((jobs_data / population_data) * 100000)

#         print(column)
#         print(jobs_data)
#         print(population_data)
#         print(jobs_per_100k)            
#         print("***///***")


        new_row[column] = jobs_per_100k  # Set the jobs_per_100k value for current year

#         print(new_row)
#         print("***///***")        
        
    # Append the new row to the list after all years have been updated with the jobs_per_100k values
    jobsper100k_new_rows.append(new_row)

In [6]:
merge_df[merge_df["geofips"]==4001]

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode
0,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,67863,67319,68072,68161,68521,69390,69602,69883,71008,71828,72181,72231,72333,71808,71026,71418,71538,71819,71920,65911,65385,population,20.0
414,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,23502,23967,24731,25532,26469,27369,29485,29453,29595,29145,28943,28618,28035,27928,27964,27843,27941,28078,27790,27089,27051,total employment,10.0


In [7]:
# Convert the list of new rows into a DataFrame
new_data = pd.DataFrame(jobsper100k_new_rows)

In [8]:
new_data

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode
0,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,34632,35602,36331,37458,38629,39442,42362,42146,41678,40576,40098,39620,38758,38893,39371,38986,39058,39096,38640,41099,41372,jobs_per_100k,30.0
1,4.0,8.0,4.0,3.0,Arizona,Cochise County,4003,42722,42220,43427,43453,44222,44456,45385,44453,43588,42901,42173,41555,40987,41030,40915,40688,40870,40561,40600,41981,41689,jobs_per_100k,30.0
2,4.0,8.0,4.0,5.0,Arizona,Coconino County,4005,56491,55807,56282,57964,58699,59486,60712,59685,57020,56707,57301,57320,58555,59279,59987,60446,61286,61000,60017,56231,59220,jobs_per_100k,30.0
3,4.0,8.0,4.0,7.0,Arizona,Gila County,4007,38813,38380,38510,38654,39429,40207,40710,40371,38771,38353,38382,39191,40054,40131,39787,39006,39018,38755,38197,38177,38845,jobs_per_100k,30.0
4,4.0,8.0,4.0,9.0,Arizona,Graham County,4009,29053,28705,29131,29701,30527,31990,33041,32992,30223,30059,30817,31696,31408,30916,31886,31913,32449,32857,32646,34091,34453,jobs_per_100k,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,4.0,8.0,56.0,37.0,Wyoming,Sweetwater County,56037,65565,63185,65554,67443,70046,74349,74328,75063,67063,67212,68814,68134,67229,66619,65058,62802,63821,64648,65044,60875,61190,jobs_per_100k,30.0
410,4.0,8.0,56.0,39.0,Wyoming,Teton County,56039,124548,122822,121352,121580,124613,128480,132029,133548,126097,123523,124204,126360,128287,131126,133046,137897,140033,144581,145474,141511,150826,jobs_per_100k,30.0
411,4.0,8.0,56.0,41.0,Wyoming,Uinta County,56041,60310,61520,60744,59897,61127,63575,65242,66764,62192,60650,59705,60163,58253,59290,58767,56921,57211,58190,58724,56157,55938,jobs_per_100k,30.0
412,4.0,8.0,56.0,43.0,Wyoming,Washakie County,56043,64849,64447,64656,65188,66031,67302,68515,69875,67244,64729,66300,65755,64453,65108,65129,64181,66167,66624,65082,66323,67090,jobs_per_100k,30.0


In [9]:
# Concatenate the new data with the original "merge_df" using pd.concat
merge_df_two = pd.concat([merge_df, new_data], ignore_index=True)

# If you want to sort the resulting DataFrame by a specific column (e.g., 'geofips' and 'year'):
merge_df_two = merge_df_two.sort_values(by=['geofips', 'description'])

# Reset the index for the updated DataFrame
merge_df_two.reset_index(drop=True, inplace=True)

In [10]:
merge_df_two[merge_df_two["geofips"]==4001]

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode
0,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,34632,35602,36331,37458,38629,39442,42362,42146,41678,40576,40098,39620,38758,38893,39371,38986,39058,39096,38640,41099,41372,jobs_per_100k,30.0
1,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,67863,67319,68072,68161,68521,69390,69602,69883,71008,71828,72181,72231,72333,71808,71026,71418,71538,71819,71920,65911,65385,population,20.0
2,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,23502,23967,24731,25532,26469,27369,29485,29453,29595,29145,28943,28618,28035,27928,27964,27843,27941,28078,27790,27089,27051,total employment,10.0


### Calculate the annual jobs growth rate for each county from 2002-2021
annual_growth_rate = (jobs_per_100k_yearY - jobs_per_100k_yearX) / jobs_per_100k_yearY

In [11]:
ann_growthrate_new_rows = []

# Iterate through unique geofips
for geofips in merge_df_two['geofips'].unique():
    # Extract data for the current geofips
    subset = merge_df_two[merge_df_two['geofips'] == geofips]

    # Create a new row with the calculated values
    new_row = subset.iloc[0].copy()
    new_row['description'] = 'ann_growthrate'
    new_row['linecode'] = 40.0    
    
    tot_growthrate_5y = 0  # Initialize the total annual growth rate for 5 years
    tot_growthrate_10y = 0  # Initialize the total annual growth rate for 10 years
    tot_growthrate_20y = 0  # Initialize the total annual growth rate for 20 years
    
    # Iterate through years to calculate annual growth rates
    for year in range(2001, 2022):
        past_column = f'yr_{year-1}'
        current_column = f'yr_{year}'

        if year == 2001:
            # Set ann_growthrate value for yr_2001 to "NA"
            new_row[current_column] = "NA"
        else:        
            # Calculate annual growth rate for the current year
            jobs_per_100k_past = subset[past_column].values[0]
            jobs_per_100k_current = subset[current_column].values[0]

            ann_growthrate = (jobs_per_100k_current - jobs_per_100k_past) / jobs_per_100k_current
            new_row[current_column] = ann_growthrate

            # Accumulate the annual growth rates
            tot_growthrate_20y += ann_growthrate
            
            if 2017 <= year <= 2021:
                tot_growthrate_5y += ann_growthrate
            if 2012 <= year <= 2021:
                tot_growthrate_10y += ann_growthrate

    # Calculate the average annual growth rates for 5 years, 10 years, and 20 years
    avg_growthrate_5y = tot_growthrate_5y / len(range(2017, 2022))
    avg_growthrate_10y = tot_growthrate_10y / len(range(2012, 2022))            

    # Calculate the average annual growth rate
    avg_growthrate_20y = tot_growthrate_20y / len(range(2002, 2022))  # Since growthrate data begins in 2002

    # Append the average annual growth rates to the new row
    new_row['avg_growthrate_5y'] = avg_growthrate_5y
    new_row['avg_growthrate_10y'] = avg_growthrate_10y
    new_row['avg_growthrate_20y'] = avg_growthrate_20y           
            
    # Append the new row to the list
    ann_growthrate_new_rows.append(new_row)

In [12]:
# Convert the list of new rows into a DataFrame
new_data = pd.DataFrame(ann_growthrate_new_rows)

# Concatenate the new data with the original "merge_df" using pd.concat
final_df = pd.concat([merge_df_two, new_data], ignore_index=True)

# If you want to sort the resulting DataFrame by a specific column (e.g., 'geofips' and 'year'):
final_df = final_df.sort_values(by=['geofips', 'description'])

# Reset the index for the updated DataFrame
final_df.reset_index(drop=True, inplace=True)

In [13]:
final_df

,region,division,state,county,stname,ctyname,geofips,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,yr_2021,description,linecode,avg_growthrate_5y,avg_growthrate_10y,avg_growthrate_20y
0,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,NA,0.027246,0.020066,0.030087,0.030314,0.020613,0.068930,-0.005125,-0.011229,-0.027159,-0.011921,-0.012065,-0.022241,0.003471,0.012141,-0.009875,0.001843,0.000972,-0.011801,0.059831,0.006599,ann_growthrate,40.0,0.011489,0.002888,0.008535
1,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,34632,35602.000000,36331.000000,37458.000000,38629.000000,39442.000000,42362.000000,42146.000000,41678.000000,40576.000000,40098.000000,39620.000000,38758.000000,38893.000000,39371.000000,38986.000000,39058.000000,39096.000000,38640.000000,41099.000000,41372.000000,jobs_per_100k,30.0,NaN,NaN,NaN
2,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,67863,67319.000000,68072.000000,68161.000000,68521.000000,69390.000000,69602.000000,69883.000000,71008.000000,71828.000000,72181.000000,72231.000000,72333.000000,71808.000000,71026.000000,71418.000000,71538.000000,71819.000000,71920.000000,65911.000000,65385.000000,population,20.0,NaN,NaN,NaN
3,4.0,8.0,4.0,1.0,Arizona,Apache County,4001,23502,23967.000000,24731.000000,25532.000000,26469.000000,27369.000000,29485.000000,29453.000000,29595.000000,29145.000000,28943.000000,28618.000000,28035.000000,27928.000000,27964.000000,27843.000000,27941.000000,28078.000000,27790.000000,27089.000000,27051.000000,total employment,10.0,NaN,NaN,NaN
4,4.0,8.0,4.0,3.0,Arizona,Cochise County,4003,NA,-0.011890,0.027794,0.000598,0.017390,0.005264,0.020469,-0.020966,-0.019845,-0.016014,-0.017262,-0.014872,-0.013858,0.001048,-0.002811,-0.005579,0.004453,-0.007618,0.000961,0.032896,-0.007004,ann_growthrate,40.0,0.004737,-0.001238,-0.001342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,4.0,8.0,56.0,43.0,Wyoming,Washakie County,56043,5232,5148.000000,5157.000000,5189.000000,5297.000000,5370.000000,5597.000000,5750.000000,5664.000000,5522.000000,5603.000000,5530.000000,5425.000000,5389.000000,5394.000000,5250.000000,5302.000000,5254.000000,5092.000000,5079.000000,5174.000000,total employment,10.0,NaN,NaN,NaN
1652,4.0,8.0,56.0,45.0,Wyoming,Weston County,56045,NA,-0.042811,-0.010397,-0.009876,0.030020,-0.011288,-0.004822,0.018298,-0.064975,0.020801,-0.023504,0.047497,-0.009848,0.041072,-0.008787,-0.014524,0.011636,-0.004648,0.010583,0.008251,0.014664,ann_growthrate,40.0,0.008097,0.009590,-0.000133
1653,4.0,8.0,56.0,45.0,Wyoming,Weston County,56045,58070,55686.000000,55113.000000,54574.000000,56263.000000,55635.000000,55368.000000,56400.000000,52959.000000,54084.000000,52842.000000,55477.000000,54936.000000,57289.000000,56790.000000,55977.000000,56636.000000,56374.000000,56977.000000,57451.000000,58306.000000,jobs_per_100k,30.0,NaN,NaN,NaN
1654,4.0,8.0,56.0,45.0,Wyoming,Weston County,56045,6487,6578.000000,6610.000000,6646.000000,6594.000000,6717.000000,7033.000000,7133.000000,7266.000000,7198.000000,7142.000000,7075.000000,7132.000000,7134.000000,7202.000000,7228.000000,6962.000000,6895.000000,6880.000000,6818.000000,6766.000000,population,20.0,NaN,NaN,NaN


In [14]:
# Create the output file path for the combined data
output_file_path = '/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/df_foranalysis.csv'

# Save the combined data to the output CSV file
final_df.to_csv(output_file_path, index=False)

In [15]:
# For testing, if necessary

# # Get a random sample of geofips identifiers
# random_seed = 427
# np.random.seed(random_seed)

# random_geofips = random.sample(final_df['geofips'].unique().tolist(), 3)

# # Create a mask to filter rows associated with the selected geofips
# mask = final_df['geofips'].isin(random_geofips)

# # Filter the DataFrame to include all rows associated with the selected geofips
# test_csv = final_df[mask].sort_values(by=['geofips', 'description'])

# # Create the output file path for the combined data
# output_file_path = '/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/test_csv.csv'

# # Save the combined data to the output CSV file
# test_csv.to_csv(output_file_path, index=False)